# Word2Vec Model

In [1]:
!pip install gensim spacy nltk

In [2]:
import gensim
import re
import spacy

In [3]:
# Download via terminal commands
!wget "https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_cbow_100_twitter.zip"
!unzip "full_grams_cbow_100_twitter.zip"

--2022-05-17 21:18:25--  https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_cbow_100_twitter.zip
Resolving bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)... 108.61.0.122, 2001:19f0:0:22::100
Connecting to bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)|108.61.0.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1131904811 (1.1G) [application/zip]
Saving to: ‘full_grams_cbow_100_twitter.zip’

full_grams_cbow_100 100%[===================>]   1.05G  36.8MB/s    in 31s     

2022-05-17 21:18:56 (34.9 MB/s) - ‘full_grams_cbow_100_twitter.zip’ saved [1131904811/1131904811]

Archive:  full_grams_cbow_100_twitter.zip
  inflating: full_grams_cbow_100_twitter.mdl  
  inflating: full_grams_cbow_100_twitter.mdl.trainables.syn1neg.npy  
  inflating: full_grams_cbow_100_twitter.mdl.wv.vectors.npy  


In [4]:
# load the AraVec model
model = gensim.models.Word2Vec.load("full_grams_cbow_100_twitter.mdl")
print("We've",len(model.wv.index2word),"vocabularies")

We've 1476715 vocabularies


# Load Dataset

In [5]:
import pandas as pd

read_file = pd.read_csv('/content/all.dev.features.txt',sep=" ",header=None)
read_file.to_csv('all_csv.csv', index=None)

In [6]:
data = pd.read_csv('/content/all_csv.csv')

In [7]:
df = data['0'].str.split(pat='\t',expand=True)
data = pd.DataFrame({'data':df[0], 'lables':df[df.shape[1]-1]})

In [8]:
data

,data,lables
0,الذرة,O
1,هي,O
2,أصغر,O
3,جزء,O
4,من,O
...,...,...
21198,BSD,I-MIS
21199,في,O
21200,طور,O
21201,التصميم,O


In [9]:
correct_labels = {
    # Incorrect classes
    "B-ERS": "B-PERS",
    "B-MSIC": "B-MISC",
    "B-OEG": "B-ORG",
    "B-ORF": "B-ORG",
    "B-PERs": "B-PERS",
    "B-PRG": "B-ORG",
    "I-PRG": "I-ORG",
    "IPERS": "I-PERS",
    "": "O",
    "o": "O",
    "B-MIS0": "B-MISC",
    "B-MIS1": "B-MISC",
    "B-MIS2": "B-MISC",
    "B-MIS3": "B-MISC",
    "I-MIS0": "I-MISC",
    "I-MIS1": "I-MISC",
    "I-MIS2": "I-MISC",
    "I-MIS3": "I-MISC",
    "B-SPANISH": "O",
    "I-SPANISH": "O",
    "B-MIS": "B-MISC",
    "I-MIS": "B-MISC",
    "B-MIS-2": "B-MISC",
    "B-MIS-1": "B-MISC",
    "B-MIS1'": "B-MISC",
    "I-MIS": "I-MISC",
    "OO": "O",
    "I--ORG": "I-ORG",
    "B-MISS1": "B-MISC",
    "IO": "O",
    "B-ENGLISH": "O",
    "B-PER": "B-PERS",
    "I-PER": "I-PERS",
    
    # Correct classes
    "B-LOC": "B-LOC",
    "O": "O",
    "B-ORG": "B-ORG",
    "I-ORG": "I-ORG",
    "B-PERS": "B-PERS",
    "I-PERS": "I-PERS",
    "I-LOC": "I-LOC",
    "I-MISC": "I-MISC",
    "B-MISC": "B-MISC",
}

In [10]:
data["lables"] = data["lables"].map(correct_labels)
# Removes the remaining unmapped classes
# Drop words that were mis-parsed from class
data.drop([item for item in data.index if pd.isnull(data["lables"][item]) or data["data"][item] in correct_labels.keys()], inplace=True)

In [11]:
data

,data,lables
0,الذرة,O
1,هي,O
2,أصغر,O
3,جزء,O
4,من,O
...,...,...
21198,BSD,I-MISC
21199,في,O
21200,طور,O
21201,التصميم,O


In [12]:
list_data=data.values.tolist()
print(list_data)

[['الذرة', 'O'], ['هي', 'O'], ['أصغر', 'O'], ['جزء', 'O'], ['من', 'O'], ['العنصر', 'O'], ['الكيميائي', 'O'], ['الذي', 'O'], ['يحتفظ', 'O'], ['بالخصائص', 'O'], ['الكيميائية', 'O'], ['لذلك', 'O'], ['العنصر', 'O'], ['.', 'O'], ['يرجع', 'O'], ['أصل', 'O'], ['كلمة', 'O'], ['ذرة', 'O'], ['إلى', 'O'], ['الكلمة', 'O'], ['الإغريقية', 'O'], ['أتوموس', 'B-MISC'], ['،', 'O'], ['وتعني', 'O'], ['غير', 'O'], ['القابل', 'O'], ['للانقسام؛', 'O'], ['إذ', 'O'], ['كان', 'O'], ['يعتقد', 'O'], ['أنه', 'O'], ['ليس', 'O'], ['ثمة', 'O'], ['ما', 'O'], ['هو', 'O'], ['أصغر', 'O'], ['من', 'O'], ['الذرة', 'O'], ['.', 'O'], ['تتكون', 'O'], ['الذرة', 'O'], ['من', 'O'], ['سحابة', 'O'], ['من', 'O'], ['الشحنات', 'O'], ['السالبة', 'O'], ['(', 'O'], ['الإلكترونات', 'B-MISC'], [')', 'O'], ['تحوم', 'O'], ['حول', 'O'], ['نواة', 'O'], ['موجبة', 'O'], ['الشحنة', 'O'], ['صغيرة', 'O'], ['جدا', 'O'], ['في', 'O'], ['الوسط', 'O'], ['.', 'O'], ['تتكون', 'O'], ['النواة', 'O'], ['الموجبة', 'O'], ['هذه', 'O'], ['من', 'O'], ['بروتونات',

In [13]:
columns = list(zip(*list_data))
word_list=list(columns[0])
lables_list=list(columns[1])

In [14]:
len(lables_list)

21156

# Data Preprocessing

In [15]:
def remove_punc(string):
    punc = '''!()-[]{};:'"\,،<>/?@#$%^&*x−×-=+_~psl1234567890'''
    for ele in string:  
        if ele in punc:  
            string = string.replace(ele, "")
            del() 
    return string

In [16]:
word_list

['الذرة',
 'هي',
 'أصغر',
 'جزء',
 'من',
 'العنصر',
 'الكيميائي',
 'الذي',
 'يحتفظ',
 'بالخصائص',
 'الكيميائية',
 'لذلك',
 'العنصر',
 '.',
 'يرجع',
 'أصل',
 'كلمة',
 'ذرة',
 'إلى',
 'الكلمة',
 'الإغريقية',
 'أتوموس',
 '،',
 'وتعني',
 'غير',
 'القابل',
 'للانقسام؛',
 'إذ',
 'كان',
 'يعتقد',
 'أنه',
 'ليس',
 'ثمة',
 'ما',
 'هو',
 'أصغر',
 'من',
 'الذرة',
 '.',
 'تتكون',
 'الذرة',
 'من',
 'سحابة',
 'من',
 'الشحنات',
 'السالبة',
 '(',
 'الإلكترونات',
 ')',
 'تحوم',
 'حول',
 'نواة',
 'موجبة',
 'الشحنة',
 'صغيرة',
 'جدا',
 'في',
 'الوسط',
 '.',
 'تتكون',
 'النواة',
 'الموجبة',
 'هذه',
 'من',
 'بروتونات',
 'موجبة',
 'الشحنة',
 '،',
 'و',
 'نيوترونات',
 'متعادلة',
 '.',
 'الذرة',
 'هي',
 'أصغر',
 'جزء',
 'من',
 'العنصر',
 'يمكن',
 'أن',
 'يتميز',
 'به',
 'عن',
 'بقية',
 'العناصر؛',
 'إذ',
 'كلما',
 'غصنا',
 'أكثر',
 'في',
 'المادة',
 'لنلاقي',
 'البنى',
 'الأصغر',
 'لن',
 'يعود',
 'هناك',
 'فرق',
 'بين',
 'عنصر',
 'و',
 'آخر',
 '.',
 'فمثلا',
 '،',
 'لا',
 'فرق',
 'بين',
 'بروتون',
 'في',
 'ذر

In [17]:
word_list = [remove_punc(i) for i in word_list]

In [18]:
word_list[22]

''

In [19]:
word_list_cleaned = []
import re
for i in range(len(word_list)):
  word_list_cleaned.append(re.sub('([@A-Za-z0-9_]+)|[^\w\s]|#|http\S+', '', word_list[i])) # cleaning up

In [20]:
lables_list_update = []
for i in range(len(lables_list)):
  if word_list_cleaned[i] != '':
    lables_list_update.append(lables_list[i])

In [21]:
word_list_cleaned = ' '.join(word_list_cleaned).split()

In [22]:
print(len(word_list_cleaned))

17985


In [23]:
len(lables_list_update)

17985

In [24]:
print(len(word_list_cleaned))

17985


In [25]:
word_list_cleaned[0]

'الذرة'

# Apply Word2vec model

In [26]:
tag_classes = ['B-LOC', 'B-MISC', 'B-ORG', 'B-PERS', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PERS', 'O']

In [27]:
model[word_list_cleaned[1]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-2.4319282e-01,  4.0879068e+00, -3.1353936e+00,  5.9956260e-02,
       -4.1635171e-01,  4.2721820e+00,  3.3861572e-05, -1.6132358e-01,
        1.9446223e+00,  1.2348921e+00, -3.0830228e+00,  1.2591254e+00,
        3.3000660e+00,  1.9770839e+00, -7.6922065e-01, -3.7119746e+00,
       -5.5515635e-01, -2.1235845e+00,  2.4543209e+00, -1.7344509e+00,
       -7.0454276e-01, -3.5545430e+00, -5.4060421e+00,  5.9016550e-01,
       -8.9479990e-02,  2.1806401e-01, -2.3470688e+00, -2.5383573e+00,
        3.3289909e-01, -3.3333282e+00, -9.8925692e-01,  3.1500992e-01,
        1.6066968e+00, -3.6605597e-01, -2.8456521e+00,  3.1417899e+00,
        3.8697834e+00,  1.6030514e+00, -4.8219749e-01,  3.3781281e-01,
        1.0049454e+00,  1.1704849e+00,  2.5190427e+00,  1.3061903e+00,
        4.1919526e-01,  4.2315874e+00,  5.8207577e-01, -6.0798913e-01,
       -1.5159211e+00,  6.3127112e-01,  6.4448071e-01,  2.1186235e+00,
       -1.5490245e+00, -8.3250850e-01,  1.3521003e+00, -2.8301327e+00,
      

In [28]:
for j, word in enumerate(word_list_cleaned):
        try:
            word_list_cleaned[j] = model[word]
        except KeyError:
            word_list_cleaned[j] = model['مجهول']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


# Saving Embedding File

In [63]:
# Save Embedding
# open file in write mode
with open(r'/content/Word2vec_embedding.txt', 'w') as fp:
    for item in word_list_cleaned:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


# One_hot_encoding_label

In [29]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
values = array(tag_classes)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
# print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# # invert first example
# inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
# print(inverted)

# using zip()
# to convert lists to dictionary
res = dict(zip(tag_classes, onehot_encoded))

['B-LOC' 'B-MISC' 'B-ORG' 'B-PERS' 'I-LOC' 'I-MISC' 'I-ORG' 'I-PERS' 'O']
[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [30]:
res

{'B-LOC': array([1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'B-MISC': array([0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 'B-ORG': array([0., 0., 1., 0., 0., 0., 0., 0., 0.]),
 'B-PERS': array([0., 0., 0., 1., 0., 0., 0., 0., 0.]),
 'I-LOC': array([0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 'I-MISC': array([0., 0., 0., 0., 0., 1., 0., 0., 0.]),
 'I-ORG': array([0., 0., 0., 0., 0., 0., 1., 0., 0.]),
 'I-PERS': array([0., 0., 0., 0., 0., 0., 0., 1., 0.]),
 'O': array([0., 0., 0., 0., 0., 0., 0., 0., 1.])}

In [31]:
res.keys()

dict_keys(['B-LOC', 'B-MISC', 'B-ORG', 'B-PERS', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PERS', 'O'])

In [32]:
One_hot_encoding_label = []
for i in range(len(lables_list_update)):
  if lables_list_update[i] in res.keys():
    One_hot_encoding_label.append(res[lables_list_update[i]])

In [33]:
One_hot_encoding_label

[array([0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0.

In [34]:
len(lables_list_update)

17985

In [35]:
len(word_list_cleaned)

17985

In [36]:
len(One_hot_encoding_label)

17985

In [37]:
import numpy as np
X = np.array(word_list_cleaned)
Y = np.array(One_hot_encoding_label)

# Data Splitting

In [38]:
from sklearn.model_selection import train_test_split
# Shuffle and split the dataset
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size=0.2)

In [39]:
len(X_Train)+len(X_Test)

17985

In [40]:
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Embedding, LSTM, GlobalMaxPooling1D, SpatialDropout1D
# batch_size = 512
# epochs = 8

# model_lstm = Sequential()
# model_lstm.add(Embedding(input_dim = max_words, output_dim = 256, input_length = max_phrase_len))
# model_lstm.add(SpatialDropout1D(0.3))
# model_lstm.add(LSTM(256, dropout = 0.3, recurrent_dropout = 0.3))
# model_lstm.add(Dense(256, activation = 'relu'))
# model_lstm.add(Dropout(0.3))
# model_lstm.add(Dense(5, activation = 'softmax'))
# model_lstm.compile(
#     loss='categorical_crossentropy',
#     optimizer='Adam',
#     metrics=['accuracy']
# )

# LSTM Model

In [54]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Input, SimpleRNN, GRU
from tensorflow.keras.layers import BatchNormalization
from keras.preprocessing.sequence import pad_sequences
from keras.layers.wrappers import TimeDistributed
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau

# crf = CRF(len(classes), input_shape=(SEQ_LENGTH, 300))
batch_size = 32
SEQ_LENGTH = 10
model = Sequential()
embedding_layer = Embedding(len(X_Train),
        100,
        weights=[X_Train],
        input_length=10,
        trainable=False)
model.add(embedding_layer)
model.add(Bidirectional(LSTM(256, return_sequences=True, activation='relu', dropout=0.5, recurrent_dropout=0.3),input_shape=(SEQ_LENGTH, 300)))
model.add(BatchNormalization())
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
reduce_lr = ReduceLROnPlateau(patience=2)
model.compile(
    loss='categorical_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

In [55]:
model.fit(X_Train, Y_Train,
          batch_size=batch_size,
          epochs=100,
          callbacks=[early_stopping],
          verbose=1
         )
with open("model.json", "w") as output:
    output.write(model.to_json())
model.save_weights("weights.hdf", overwrite=True)

Epoch 1/100


ValueError: ignored

# Precision & Recall

In [ ]:
Y_Pred = model.predict_classes(X_Test, verbose=0)


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

print('Precision: %.3f' % precision_score(Y_Test, Y_Pred))

print('Recall: %.3f' % recall_score(Y_Test, Y_Pred))

print('Accuracy: %.3f' % accuracy_score(Y_Test, Y_Pred))

print('F1 Score: %.3f' % f1_score(Y_Test, Y_Pred))